In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from utilities_namespace import *
from helpers.notebooks import skip_on_import

In [2]:
from functools import partial

## Scoring functions

In [3]:
from signature_scoring.scoring_functions.gsea import create_gsea_scorer
from signature_scoring.scoring_functions.limma import create_roast_scorer
from signature_scoring.scoring_functions.gsva import create_gsva_scorer
from signature_scoring.scoring_functions.generic_scorers import (
    score_spearman, score_spearman_max,
    x_sum, x_sum_max,
    x_product, x_product_max,
    x_cos
)

In [4]:
from signature_scoring.scoring_functions import scoring_function

In [5]:
from random import normalvariate

# single sample version
@scoring_function
def ss_random_normal_score(a, b):
    # NB: gauss might be faster, but normalvariate is thread-safe
    # 1.3 µs
    # %timeit normalvariate(0, 1)
    # from numpy.random import normal
    # 3.57 µs
    # %timeit normal()
    # from random import gauss
    # %timeit gauss(0, 1)
    # 888 ns
    return normalvariate(0, 1)

In [6]:
from signature_scoring.models.with_controls import ExpressionWithControls

# multi sample version
def ms_random_normal_score(a, b):
    return normalvariate(0, 1)

ms_random_normal_score = scoring_function(
    ms_random_normal_score, input=ExpressionWithControls,
    grouping='by_substance'
)

### GSEA

Only for comparisons:

In [7]:
gsea_score = create_gsea_scorer(permutations=1000, gene_sets='c2.cp.kegg', q_value_cutoff=0.05)
gsea_score_reactome = create_gsea_scorer(permutations=1000, gene_sets='c2.cp.reactome', q_value_cutoff=0.05)
gsea_score_hallmarks = create_gsea_scorer(permutations=1000, gene_sets='h.all', q_value_cutoff=0.05)

In [8]:
from methods.gsea import cudaGSEA

In [9]:
cutoff = 0.5

#### Please note that this benchmarking function trims gene-sets to genes tested TCGA 

In [10]:
from data_sources.tcga import TCGA
tcga = TCGA()

In [11]:
brca_genes = set(tcga.expression.genes('BRCA'))

In [12]:
%%skip_on_import
for cohort in tcga.expression.cohorts():
    cohort_genes = tcga.expression.genes(cohort)
    if brca_genes != set(cohort_genes):
        raise Exception(cohort, 'had different genes tested than BRCA')
else:
    print('All cohorts have same genes')

All cohorts have same genes


In [13]:
tcga_genes = brca_genes

In [14]:
cuda = partial(
    create_gsea_scorer,
    gsea_app=cudaGSEA(),
    permutations=1000, q_value_cutoff=0.5,
    permutation_type='phenotype', grouping='by_substance',
    metric='twopass_signal2noise',
    verbose=False,
    genes=tcga_genes
)

In [15]:
gsea_score_phenotypes_cuda_kegg = cuda(gene_sets='c2.cp.kegg')
gsea_score_phenotypes_cuda_reactome = cuda(gene_sets='c2.cp.reactome')
gsea_score_phenotypes_cuda_hallmarks = cuda(gene_sets='h.all')

### Limma

In [16]:
roast_score_by_substance = create_roast_scorer(gene_sets='c2.cp.kegg', q_value_cutoff=cutoff)

### GSVA

In [17]:
gsva_score_by_substance = create_gsva_scorer(gene_sets='c2.cp.kegg', q_value_cutoff=cutoff)
zscore_by_substance = create_gsva_scorer(gene_sets='c2.cp.kegg', q_value_cutoff=cutoff, method='zscore')
plage_score_by_substance = create_gsva_scorer(gene_sets='c2.cp.kegg', q_value_cutoff=cutoff, method='plage')
ssgsea_score_by_substance = create_gsva_scorer(gene_sets='c2.cp.kegg', q_value_cutoff=cutoff, method='ssgsea')

In [18]:
single_sample_gsva = partial(
    create_gsva_scorer, gene_sets='c2.cp.kegg', q_value_cutoff=cutoff,
    single_sample=True, grouping=None, permutations=100
)

gsva_score = single_sample_gsva(method='gsva')
ssgsea_score = single_sample_gsva(method='ssgsea')
plage_score = single_sample_gsva(method='plage')
zscore = single_sample_gsva(method='zscore')

### Connectivity score derived

In [19]:
import signature_scoring.scoring_functions.connectivity_score as connectivity

In [20]:
connectivity_score_disease = connectivity.create_scorer(
    negative=True,
    ranks_type='disease'
)

negative=True: substances which might treat disease of given profile will get high scores


In [21]:
# - Reverse Gene Expression Score
connectivity_score_RGES = connectivity.create_scorer(
    negative=True,
    ranks_type='signature',
    statistic=connectivity.kolmogorov_smirnov,
    compose_tags=connectivity.difference
)

negative=True: substances which might treat disease of given profile will get high scores


In [22]:
show_source(connectivity_score_RGES)

In [23]:
# as tested in (Cheng, 2014) study
connectivity_score = connectivity.create_scorer(
    negative=True,
    ranks_type='signature',   
    statistic=connectivity.kolmogorov_smirnov,
    compose_tags=connectivity.conditional_difference
)
show_source(connectivity_score)

negative=True: substances which might treat disease of given profile will get high scores


In [24]:
connectivity_score_cramér_von_mises = connectivity.create_scorer(
    negative=False,
    ranks_type='signature',
    statistic=connectivity.cramér_von_mises,
    compose_tags=connectivity.difference
)
show_source(connectivity_score_cramér_von_mises)

negative=False: substances with similar effect on expression will get high scores


### All tested

In [25]:
single_sample_gene_set_based = [
    # single-sample gsva: too expensive computionally
    plage_score, gsva_score, ssgsea_score, zscore,

    # GSEADesktop - just too slow
    gsea_score, gsea_score_reactome, gsea_score_hallmarks
]

# these should be described in a Supplementary Text
disregarded = [
    # this was an experiment, but it does not have a sound reasoning behind
    connectivity_score_disease,
    
    # here the idea was that looking at the most extreme effects alone may be enough;
    # actually it was not bad - the results were quite similar, but it does not contribute
    # towards the goal in any way (except for proving that this intuition is not so bad)
    x_product_max, x_sum_max, score_spearman_max
]

tested_single_sample_functions = [
    # simple & fast
    score_spearman,
    x_sum, x_product, x_cos,

    connectivity_score,
    connectivity_score_RGES,
    connectivity_score_cramér_von_mises,
    connectivity.pharmaco_gx_connectivity_score
]

tested_multi_sample_functions = [
    # gsva
    plage_score_by_substance, gsva_score_by_substance,
    ssgsea_score_by_substance, zscore_by_substance,

    # limma
    roast_score_by_substance,
    
    # cudaGSEA
    gsea_score_phenotypes_cuda_kegg,
    gsea_score_phenotypes_cuda_reactome,
    gsea_score_phenotypes_cuda_hallmarks
]

tested_scoring_functions = (
    tested_single_sample_functions
    +
    tested_multi_sample_functions
)

## Benchmarking

In [26]:
from signature_scoring.evaluation.benchmark import benchmark
show_source(benchmark)

## Benchmarking set-up shared across tests

Following values were set as defaults for the test on BRCA cohort:

In [27]:
standard_benchmark = partial(
    benchmark,
    top='quantile',                           # criterium for selection of the candidated repositioning substances; quantile: choose top 10%
    summary='per_cell_line_combined',         # use combine() method of evaluation metrics to combine metrics results calculated for each of the cell lines
    aggregate='mean_per_substance_and_dose',  # same as "mean_per_substance_dose_and_cell" because summary is computed per cell line anyway (see above)
    cell_lines_ratio=.75,                     # at least such a ratio of substances signatures should be available for the cell line to include it
    limit=500,                                # how many under- and over- expressed genes should be analyzed
    processes=8,
    per_test_progress=False
)